### ERA5 Extraction

In [13]:
import ee 
import pandas as pd
import numpy as np
import os
from datetime import datetime
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
%cd ..

/Users/youngoh/Desktop/flood_urop_repo/mml_flood


/Users/youngoh/Desktop/flood_urop_repo/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
ee.Authenticate()
ee.Initialize(project='my-project-410920') ### YOUR PROJECT HERE

In [3]:
variables_to_extract = ['temperature_2m', 'total_precipitation_sum', 'u_component_of_wind_10m',
						'v_component_of_wind_10m', 'surface_pressure', 'snowfall_sum',
						'snowmelt_sum', 'dewpoint_temperature_2m',
    					]

In [16]:
xy_df_sub1 = pd.read_csv('data/testing/x_stat.csv')
y_master = pd.read_csv('data/testing/y_master.csv')
n_pred = 1
# lookup = pd.read_csv('lookup.csv')


In [17]:
def attach_target(x_df, y_master, disaster, next_n):
    """
    Attach a 'target' column to x_df based on disaster data in y_master.
    If no data exists in y_master for a given grid_id and year, set the target to NaN.
    Look up from next 1 to next_n years, if there is a major flood occuring.
    """
    
    x_df = x_df.copy()
    # Create a dictionary for fast lookup: {(grid_id, year): disaster_value}
    disaster_lookup = {
        (row['grid_id'], row['year']): row[disaster+'_bin']
        for _, row in y_master.iterrows()
    }

    # Initialize a 'target' column in x_df
    target_col = 'target_' + disaster + '_' + str(next_n)
    x_df[target_col] = np.nan  # Default to NaN

    # Iterate over x_df rows
    for idx, row in x_df.iterrows():
        grid_id = row['grid_id']
        year = row['year']

        # Check if (grid_id, year + next_n + 1) exists in y_master
        if (grid_id, year + next_n ) not in disaster_lookup:
            # No data found for (grid_id, year + next_n + 1), skip this row
            continue

        # Check years from year+1 to year+next_n
        target_found = 0
        for i in range(1, next_n + 1):
            future_year = year + i
            if disaster_lookup.get((grid_id, future_year), 0) == 1:
                target_found = 1
                break

        # Update the 'target' column
        x_df.at[idx, target_col] = target_found
    # Drop rows where 'target' is NaN
    x_df = x_df.dropna(subset=[target_col])

    return x_df


In [18]:
xy_df_sub1 = attach_target(xy_df_sub1, y_master, 'flood', n_pred)

xy_df_sub1

,Unnamed: 0,grid_id,year,stat_flood_amt,stat_storm_amt,stat_earthquake_amt,stat_extreme temperature _amt,stat_landslide_amt,stat_volcanic activity_amt,stat_drought_amt,...,stat_storm_bin,stat_earthquake_bin,stat_extreme temperature _bin,stat_landslide_bin,stat_volcanic activity_bin,stat_drought_bin,stat_mass movement (dry)_bin,stat_lat,stat_lon,target_flood_1
0,0,"(-46, 168)",1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46,168,0.0
1,1,"(-46, 168)",1961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46,168,0.0
2,2,"(-46, 168)",1962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46,168,0.0
3,3,"(-46, 168)",1963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46,168,0.0
4,4,"(-46, 168)",1964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-46,168,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48964,48964,"(66, 130)",2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66,130,0.0
48965,48965,"(66, 130)",2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66,130,0.0
48966,48966,"(66, 130)",2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66,130,0.0
48967,48967,"(66, 130)",2016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66,130,0.0


In [51]:
def create_directories(row, path):
    '''
    create directories that will be populated according to the dataset
    '''
    flood_target = row[f'target_flood_{n_pred}']

    # Sanitize the row name to create a valid folder name
    directory_name = f'extracted_{row.name}'
    directory_name = directory_name.replace('/', '_').replace(' ', '_')

    # Create the directory for this row, ensuring it's valid
    if flood_target == 1:
        output_dir = os.path.join(f'{path}/target_flood', directory_name)
    else:
        output_dir = os.path.join(f'{path}/no_flood_target', directory_name)

    os.makedirs(output_dir, exist_ok=True)

# Ensure directories are created before running the multithreaded execution
path = f'data/era5/target_{n_pred}_redo'
for _, row in tqdm(xy_df_sub1.iterrows(), total=len(xy_df_sub1)):
    create_directories(row, path)

100%|██████████| 48970/48970 [00:03<00:00, 14097.40it/s]


In [57]:
# Load the Earth Engine ImageCollection
IC = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR').select(variables_to_extract)

def process_grid_point(args):
    '''
    process entries of xy_df_sub1 and populate appropriate directories with raw era5 data
    '''
    index, row = args
    # latitude_to_extract = row['stat_latitude']
    # longitude_to_extract = row['stat_longitude']
    lat, lon = row['grid_id'].split(', ')
    latitude_to_extract = float(lat[1:])
    longitude_to_extract = float(lon[:-1])

    year_to_process = int(row['year'])
    grid_id = row['grid_id']
    flood_target = row[f'target_flood_{n_pred}']
    point = ee.Geometry.Point(longitude_to_extract, latitude_to_extract)

    # Create the output directory
    output_dir = f'{path}/target_flood/extracted_{index}' if flood_target == 1 else f'{path}/no_flood_target/extracted_{index}'
    os.makedirs(output_dir, exist_ok=True)

    # Define the date range for the entire period (5 years)
    start_date = datetime(year_to_process - 4, 1, 1)
    end_date = datetime(year_to_process, 12, 31)

    # Filter the ImageCollection for the entire date range
    era5_tp = IC.filterDate(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

    # Extract the time series for the entire period
    try:
        time_series = era5_tp.getRegion(point, scale=100000).getInfo()
        time_series_df = pd.DataFrame(time_series[1:], columns=time_series[0])

        # Convert the time column to a readable date format
        time_series_df['time'] = pd.to_datetime(time_series_df['time'], unit='ms')

        # Melt the DataFrame to the desired format
        melted_df = time_series_df.melt(
            id_vars=['time'],
            value_vars=variables_to_extract,
            var_name='id',
            value_name='value'
        )

        # Sort by variable name (id) and then by time
        melted_df = melted_df.sort_values(by=['id', 'time'])

        # Save the entire melted DataFrame to a single CSV file
        output_file = os.path.join(output_dir, f'era5_data.csv')
        melted_df.to_csv(output_file, index=False)

    except Exception as e:
        print(f"Failed to process grid point {grid_id, row.name, point, longitude_to_extract, latitude_to_extract}: {e}")


In [53]:
with ThreadPoolExecutor(max_workers=8) as executor:
	list(tqdm(executor.map(process_grid_point, [(index, row) for index, row in xy_df_sub1.iterrows()]), total=len(xy_df_sub1)))

100%|██████████| 48970/48970 [17:35<00:00, 46.38it/s]


In [54]:
def populate_dir(xy_df_sub1, path):
	'''
	read directories and flatten weather vars
	'''
	directories = sorted([f for f in os.listdir(path) if not f.startswith('.')])
	num_features = 480

	# Create column names for the new features
	column_names = [f'era_{i}' for i in range(num_features)]

	# Create an empty list to store the rows of data
	data = []

	# Loop through directories and process each one
	for directory in tqdm(directories):
		# Extract index from directory name
		idx = int(directory.split('_')[1])

		# Read and flatten features
		features_df = pd.read_csv(f'{path}/{directory}/era5_data.csv')
		values = list(features_df['value'])

		# Gather the entire row's data (keep all existing columns)
		row_data = xy_df_sub1.loc[idx].values  # All columns in that row

		# Append the original row data + the new flattened feature values
		data.append(list(row_data) + list(values))

	# Create a new DataFrame with all original columns and the new features
	return pd.DataFrame(data, columns=xy_df_sub1.columns.tolist() + column_names)

In [55]:
xy_df_sub_new_no_flood = populate_dir(xy_df_sub1, f'{path}/no_flood_target')
xy_df_sub_new_flood = populate_dir(xy_df_sub1, f'{path}/target_flood')

100%|██████████| 11666/11666 [00:07<00:00, 1583.22it/s]


In [56]:
xy_df_sub1_new_combined = pd.concat([xy_df_sub_new_no_flood, xy_df_sub_new_flood], axis=0)
xy_df_sub1_new_combined.to_csv('data/xy_df_renamed/xy_df_sub5_full.csv', index=True)